In [176]:
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt 
import seaborn as sns 
from sklearn.model_selection import GridSearchCV,train_test_split
from sklearn.ensemble import VotingClassifier,RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report,accuracy_score,recall_score,f1_score,precision_score
from sklearn.preprocessing import StandardScaler,LabelEncoder
from sklearn.neighbors import KNeighborsClassifier
from mlxtend.plotting import plot_decision_regions

In [21]:
df = pd.read_csv("F:/dataset_for_ML/Ensemble Learning/classification/breastcancerpredictionwinconsin.csv")

In [22]:
df.head()

,id,diagnosis,radius_mean,texture_mean,perimeter_mean,area_mean,smoothness_mean,compactness_mean,concavity_mean,concave points_mean,...,texture_worst,perimeter_worst,area_worst,smoothness_worst,compactness_worst,concavity_worst,concave points_worst,symmetry_worst,fractal_dimension_worst,Unnamed: 32
0,842302,M,17.99,10.38,122.80,1001.0,0.11840,0.27760,0.3001,0.14710,...,17.33,184.60,2019.0,0.1622,0.6656,0.7119,0.2654,0.4601,0.11890,NaN
1,842517,M,20.57,17.77,132.90,1326.0,0.08474,0.07864,0.0869,0.07017,...,23.41,158.80,1956.0,0.1238,0.1866,0.2416,0.1860,0.2750,0.08902,NaN
2,84300903,M,19.69,21.25,130.00,1203.0,0.10960,0.15990,0.1974,0.12790,...,25.53,152.50,1709.0,0.1444,0.4245,0.4504,0.2430,0.3613,0.08758,NaN
3,84348301,M,11.42,20.38,77.58,386.1,0.14250,0.28390,0.2414,0.10520,...,26.50,98.87,567.7,0.2098,0.8663,0.6869,0.2575,0.6638,0.17300,NaN
4,84358402,M,20.29,14.34,135.10,1297.0,0.10030,0.13280,0.1980,0.10430,...,16.67,152.20,1575.0,0.1374,0.2050,0.4000,0.1625,0.2364,0.07678,NaN


In [23]:
df.shape

(569, 33)

In [24]:
df.isna().sum()

id                           0
diagnosis                    0
radius_mean                  0
texture_mean                 0
perimeter_mean               0
area_mean                    0
smoothness_mean              0
compactness_mean             0
concavity_mean               0
concave points_mean          0
symmetry_mean                0
fractal_dimension_mean       0
radius_se                    0
texture_se                   0
perimeter_se                 0
area_se                      0
smoothness_se                0
compactness_se               0
concavity_se                 0
concave points_se            0
symmetry_se                  0
fractal_dimension_se         0
radius_worst                 0
texture_worst                0
perimeter_worst              0
area_worst                   0
smoothness_worst             0
compactness_worst            0
concavity_worst              0
concave points_worst         0
symmetry_worst               0
fractal_dimension_worst      0
Unnamed:

In [25]:
df.drop(columns=["Unnamed: 32","id"],inplace=True)

In [26]:
df.duplicated().sum()

0

In [27]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 569 entries, 0 to 568
Data columns (total 31 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   diagnosis                569 non-null    object 
 1   radius_mean              569 non-null    float64
 2   texture_mean             569 non-null    float64
 3   perimeter_mean           569 non-null    float64
 4   area_mean                569 non-null    float64
 5   smoothness_mean          569 non-null    float64
 6   compactness_mean         569 non-null    float64
 7   concavity_mean           569 non-null    float64
 8   concave points_mean      569 non-null    float64
 9   symmetry_mean            569 non-null    float64
 10  fractal_dimension_mean   569 non-null    float64
 11  radius_se                569 non-null    float64
 12  texture_se               569 non-null    float64
 13  perimeter_se             569 non-null    float64
 14  area_se                  5

In [28]:
df.select_dtypes(["object"])

,diagnosis
0,M
1,M
2,M
3,M
4,M
...,...
564,M
565,M
566,M
567,M


In [42]:
X = df.iloc[:,1:]
y = df["diagnosis"]

In [45]:
X.describe(include='all')

,radius_mean,texture_mean,perimeter_mean,area_mean,smoothness_mean,compactness_mean,concavity_mean,concave points_mean,symmetry_mean,fractal_dimension_mean,...,radius_worst,texture_worst,perimeter_worst,area_worst,smoothness_worst,compactness_worst,concavity_worst,concave points_worst,symmetry_worst,fractal_dimension_worst
count,569.000000,569.000000,569.000000,569.000000,569.000000,569.000000,569.000000,569.000000,569.000000,569.000000,...,569.000000,569.000000,569.000000,569.000000,569.000000,569.000000,569.000000,569.000000,569.000000,569.000000
mean,14.127292,19.289649,91.969033,654.889104,0.096360,0.104341,0.088799,0.048919,0.181162,0.062798,...,16.269190,25.677223,107.261213,880.583128,0.132369,0.254265,0.272188,0.114606,0.290076,0.083946
std,3.524049,4.301036,24.298981,351.914129,0.014064,0.052813,0.079720,0.038803,0.027414,0.007060,...,4.833242,6.146258,33.602542,569.356993,0.022832,0.157336,0.208624,0.065732,0.061867,0.018061
min,6.981000,9.710000,43.790000,143.500000,0.052630,0.019380,0.000000,0.000000,0.106000,0.049960,...,7.930000,12.020000,50.410000,185.200000,0.071170,0.027290,0.000000,0.000000,0.156500,0.055040
25%,11.700000,16.170000,75.170000,420.300000,0.086370,0.064920,0.029560,0.020310,0.161900,0.057700,...,13.010000,21.080000,84.110000,515.300000,0.116600,0.147200,0.114500,0.064930,0.250400,0.071460
50%,13.370000,18.840000,86.240000,551.100000,0.095870,0.092630,0.061540,0.033500,0.179200,0.061540,...,14.970000,25.410000,97.660000,686.500000,0.131300,0.211900,0.226700,0.099930,0.282200,0.080040
75%,15.780000,21.800000,104.100000,782.700000,0.105300,0.130400,0.130700,0.074000,0.195700,0.066120,...,18.790000,29.720000,125.400000,1084.000000,0.146000,0.339100,0.382900,0.161400,0.317900,0.092080
max,28.110000,39.280000,188.500000,2501.000000,0.163400,0.345400,0.426800,0.201200,0.304000,0.097440,...,36.040000,49.540000,251.200000,4254.000000,0.222600,1.058000,1.252000,0.291000,0.663800,0.207500


In [65]:
scaling = StandardScaler()

In [67]:
X_new = pd.DataFrame(scaling.fit_transform(X),columns=X.columns)

In [68]:
X_new

,radius_mean,texture_mean,perimeter_mean,area_mean,smoothness_mean,compactness_mean,concavity_mean,concave points_mean,symmetry_mean,fractal_dimension_mean,...,radius_worst,texture_worst,perimeter_worst,area_worst,smoothness_worst,compactness_worst,concavity_worst,concave points_worst,symmetry_worst,fractal_dimension_worst
0,1.097064,-2.073335,1.269934,0.984375,1.568466,3.283515,2.652874,2.532475,2.217515,2.255747,...,1.886690,-1.359293,2.303601,2.001237,1.307686,2.616665,2.109526,2.296076,2.750622,1.937015
1,1.829821,-0.353632,1.685955,1.908708,-0.826962,-0.487072,-0.023846,0.548144,0.001392,-0.868652,...,1.805927,-0.369203,1.535126,1.890489,-0.375612,-0.430444,-0.146749,1.087084,-0.243890,0.281190
2,1.579888,0.456187,1.566503,1.558884,0.942210,1.052926,1.363478,2.037231,0.939685,-0.398008,...,1.511870,-0.023974,1.347475,1.456285,0.527407,1.082932,0.854974,1.955000,1.152255,0.201391
3,-0.768909,0.253732,-0.592687,-0.764464,3.283553,3.402909,1.915897,1.451707,2.867383,4.910919,...,-0.281464,0.133984,-0.249939,-0.550021,3.394275,3.893397,1.989588,2.175786,6.046041,4.935010
4,1.750297,-1.151816,1.776573,1.826229,0.280372,0.539340,1.371011,1.428493,-0.009560,-0.562450,...,1.298575,-1.466770,1.338539,1.220724,0.220556,-0.313395,0.613179,0.729259,-0.868353,-0.397100
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
564,2.110995,0.721473,2.060786,2.343856,1.041842,0.219060,1.947285,2.320965,-0.312589,-0.931027,...,1.901185,0.117700,1.752563,2.015301,0.378365,-0.273318,0.664512,1.629151,-1.360158,-0.709091
565,1.704854,2.085134,1.615931,1.723842,0.102458,-0.017833,0.693043,1.263669,-0.217664,-1.058611,...,1.536720,2.047399,1.421940,1.494959,-0.691230,-0.394820,0.236573,0.733827,-0.531855,-0.973978
566,0.702284,2.045574,0.672676,0.577953,-0.840484,-0.038680,0.046588,0.105777,-0.809117,-0.895587,...,0.561361,1.374854,0.579001,0.427906,-0.809587,0.350735,0.326767,0.414069,-1.104549,-0.318409
567,1.838341,2.336457,1.982524,1.735218,1.525767,3.272144,3.296944,2.658866,2.137194,1.043695,...,1.961239,2.237926,2.303601,1.653171,1.430427,3.904848,3.197605,2.289985,1.919083,2.219635


In [76]:
X_train,X_test,y_train,y_test = train_test_split(X_new,y,random_state=42,test_size=0.2)

Just working with normal Logistic Regression

In [70]:
lr = LogisticRegression()

In [71]:
lr.fit(X_train,y_train)

LogisticRegression()

In [72]:
pred  = lr.predict(X_test)

In [77]:
print(classification_report(pred,y_test)),print(accuracy_score(pred,y_test))

              precision    recall  f1-score   support

           B       0.99      0.97      0.98        72
           M       0.95      0.98      0.96        42

    accuracy                           0.97       114
   macro avg       0.97      0.97      0.97       114
weighted avg       0.97      0.97      0.97       114

0.9736842105263158


(None, None)

Just a normal Decision Tree Classifier

In [85]:
dtc = DecisionTreeClassifier(random_state=42)

In [86]:
dtc.fit(X_train,y_train)

DecisionTreeClassifier(random_state=42)

In [87]:
pred_dtc = dtc.predict(X_test)

In [88]:
print(classification_report(pred_dtc,y_test)),print(accuracy_score(pred_dtc,y_test))

              precision    recall  f1-score   support

           B       0.96      0.96      0.96        71
           M       0.93      0.93      0.93        43

    accuracy                           0.95       114
   macro avg       0.94      0.94      0.94       114
weighted avg       0.95      0.95      0.95       114

0.9473684210526315


(None, None)

In [113]:
param_grid = {
    "criterion" : ["gini","entropy","log_loss"],
    "max_depth" : [None,1,2,4,6,8,10],
    "min_samples_split" : [None,1,2,3,5,7],
    "min_samples_leaf" : [None,1,2,3,5,7],
    "max_features" : [0.1,0.2,0.3,0.5,0.7,1.0],
    "max_leaf_nodes" : [None,1,2,5]
}

In [114]:
dtc_tuned = DecisionTreeClassifier(random_state=42)

In [115]:
gridSearch_dt = GridSearchCV(dtc_tuned,param_grid,scoring="accuracy",n_jobs=-1)

In [116]:
gridSearch_dt.fit(X_train,y_train)

c:\Users\DELL\.conda\envs\pandasNumpyMatplotlibPrac\lib\site-packages\sklearn\model_selection\_validation.py:528: FitFailedWarning: 
52920 fits failed out of a total of 90720.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
11340 fits failed with the following error:
Traceback (most recent call last):
  File "c:\Users\DELL\.conda\envs\pandasNumpyMatplotlibPrac\lib\site-packages\sklearn\model_selection\_validation.py", line 866, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "c:\Users\DELL\.conda\envs\pandasNumpyMatplotlibPrac\lib\site-packages\sklearn\base.py", line 1382, in wrapper
    estimator._validate_params()
  File "c:\Users\DELL\.conda\envs\pandasNumpyMatplotlibPrac\lib\site-packages\sklearn\base.py", 

GridSearchCV(estimator=DecisionTreeClassifier(random_state=42), n_jobs=-1,
             param_grid={'criterion': ['gini', 'entropy', 'log_loss'],
                         'max_depth': [None, 1, 2, 4, 6, 8, 10],
                         'max_features': [0.1, 0.2, 0.3, 0.5, 0.7, 1.0],
                         'max_leaf_nodes': [None, 1, 2, 5],
                         'min_samples_leaf': [None, 1, 2, 3, 5, 7],
                         'min_samples_split': [None, 1, 2, 3, 5, 7]},
             scoring='accuracy')

In [118]:
gridSearch_dt.best_score_,gridSearch_dt.best_params_

(0.9626373626373625,
 {'criterion': 'gini',
  'max_depth': None,
  'max_features': 0.1,
  'max_leaf_nodes': None,
  'min_samples_leaf': 5,
  'min_samples_split': 2})

In [119]:
pred_dt_grid = gridSearch_dt.predict(X_test)

In [121]:
print(classification_report(pred_dt_grid,y_test)),accuracy_score(pred_dt_grid,y_test)

              precision    recall  f1-score   support

           B       0.93      0.94      0.94        70
           M       0.91      0.89      0.90        44

    accuracy                           0.92       114
   macro avg       0.92      0.91      0.92       114
weighted avg       0.92      0.92      0.92       114



(None, 0.9210526315789473)

using svc with different kernels 

In [190]:
svc_lr = SVC(kernel="linear",probability=True)
svc_pl = SVC(kernel="poly",probability=True)
svc_rb = SVC(kernel="rbf",probability=True)

In [191]:
svc_lr.fit(X_train,y_train)
svc_pl.fit(X_train,y_train)
svc_rb.fit(X_train,y_train)

SVC(probability=True)

In [192]:
pred_svc_lr =  svc_lr.predict(X_test)
pred_svc_pl =  svc_pl.predict(X_test)
pred_svc_rb =  svc_rb.predict(X_test)

In [193]:
print("classification Report SVC(linear) \n",classification_report(pred_svc_lr,y_test))
print("classification Report SVC(poly) \n",classification_report(pred_svc_pl,y_test))
print("classification Report SVC(rb) \n",classification_report(pred_svc_rb,y_test))

classification Report SVC(linear) 
               precision    recall  f1-score   support

           B       0.96      0.97      0.96        70
           M       0.95      0.93      0.94        44

    accuracy                           0.96       114
   macro avg       0.96      0.95      0.95       114
weighted avg       0.96      0.96      0.96       114

classification Report SVC(poly) 
               precision    recall  f1-score   support

           B       1.00      0.83      0.90        86
           M       0.65      1.00      0.79        28

    accuracy                           0.87       114
   macro avg       0.83      0.91      0.85       114
weighted avg       0.91      0.87      0.88       114

classification Report SVC(rb) 
               precision    recall  f1-score   support

           B       0.99      0.97      0.98        72
           M       0.95      0.98      0.96        42

    accuracy                           0.97       114
   macro avg       0.97   

In [194]:
print("accuracy score SVC(linear)",accuracy_score(pred_svc_lr,y_test))
print("accuracy score SVC(poly)",accuracy_score(pred_svc_pl,y_test))
print("accuracy score SVC(rb)",accuracy_score(pred_svc_rb,y_test))

accuracy score SVC(linear) 0.956140350877193
accuracy score SVC(poly) 0.868421052631579
accuracy score SVC(rb) 0.9736842105263158


USING RANDOM FOREST,(NORMAL)

In [195]:
# le = LabelEncoder()
# y_train_en = le.fit_transform(y_train)
# y_test_en = le.fit_transform(y_test)

In [196]:
rf = RandomForestClassifier(random_state=42)

In [197]:
rf.fit(X_train,y_train)

RandomForestClassifier(random_state=42)

In [198]:
rf_pred = rf.predict(X_test)

In [199]:
print(classification_report(rf_pred,y_test)),print(accuracy_score(rf_pred,y_test))

              precision    recall  f1-score   support

           B       0.99      0.96      0.97        73
           M       0.93      0.98      0.95        41

    accuracy                           0.96       114
   macro avg       0.96      0.97      0.96       114
weighted avg       0.97      0.96      0.97       114

0.9649122807017544


(None, None)

In [200]:
param_grid_rf = {
    'n_estimators': [50, 100, 200],
    'max_depth': [2, 5, 10, None],
    'min_samples_split': [2, 5],
    'max_features': ['sqrt', 'log2'],
    'bootstrap': [True, False]  
}

In [201]:
gridSearch_rf = GridSearchCV(rf,param_grid_rf,n_jobs=-1,cv=5,scoring="accuracy")

In [202]:
gridSearch_rf.fit(X_train,y_train)

GridSearchCV(cv=5, estimator=RandomForestClassifier(random_state=42), n_jobs=-1,
             param_grid={'bootstrap': [True, False],
                         'max_depth': [2, 5, 10, None],
                         'max_features': ['sqrt', 'log2'],
                         'min_samples_split': [2, 5],
                         'n_estimators': [50, 100, 200]},
             scoring='accuracy')

In [203]:
gridSearch_rf.best_params_,gridSearch_rf.best_score_

({'bootstrap': True,
  'max_depth': 10,
  'max_features': 'log2',
  'min_samples_split': 5,
  'n_estimators': 50},
 0.9670329670329669)

In [204]:
knn = KNeighborsClassifier(n_neighbors=5)
knn.fit(X_train,y_train)

KNeighborsClassifier()

In [205]:
knn_pred = knn.predict(X_test)

In [206]:
print(classification_report(knn_pred,y_test)),print(accuracy_score(knn_pred,y_test))

              precision    recall  f1-score   support

           B       0.96      0.96      0.96        71
           M       0.93      0.93      0.93        43

    accuracy                           0.95       114
   macro avg       0.94      0.94      0.94       114
weighted avg       0.95      0.95      0.95       114

0.9473684210526315


(None, None)

USING VOTING ENSEMBLE FOR ALL THE ABOVE ALGO ! 

In [207]:
models = [('rfc',rf),("rfc_param",gridSearch_rf),('dtc',dtc),("dtc_param",gridSearch_dt),('lr',lr),('knn',knn),('svm_lr',svc_lr),('svm_pl',svc_pl),('svm_rb',svc_rb)] 

In [208]:
vc_soft = VotingClassifier(models,voting="soft",n_jobs=-1)
vc_hard = VotingClassifier(models,voting="hard",n_jobs=-1)

In [209]:
vc_soft.fit(X_train,y_train)
vc_hard.fit(X_train,y_train)

VotingClassifier(estimators=[('rfc', RandomForestClassifier(random_state=42)),
                             ('rfc_param',
                              GridSearchCV(cv=5,
                                           estimator=RandomForestClassifier(random_state=42),
                                           n_jobs=-1,
                                           param_grid={'bootstrap': [True,
                                                                     False],
                                                       'max_depth': [2, 5, 10,
                                                                     None],
                                                       'max_features': ['sqrt',
                                                                        'log2'],
                                                       'min_samples_split': [2,
                                                                             5],
                                                       'n_estimators': [50, 100,
                                                                        200]},
                                           scoring='accuracy')),
                             ('dtc', Decis...
                                                       'max_features': [0.1,
                                                                        0.2,
                                                                        0.3,
                                                                        0.5,
                                                                        0.7,
                                                                        1.0],
                                                       'max_leaf_nodes': [None,
                                                                          1, 2,
                                                                          5],
                                                       'min_samples_leaf': [None,
                                                                            1,
                                                                            2,
                                                                            3,
                                                                            5,
                                                                            7],
                                                       'min_samples_split': [None,
                                                                             1,
                                                                             2,
                                                                             3,
                                                                             5,
                                                                             7]},
                                           scoring='accuracy')),
                             ('lr', LogisticRegression()),
                             ('knn', KNeighborsClassifier()),
                             ('svm_lr', SVC(kernel='linear', probability=True)),
                             ('svm_pl', SVC(kernel='poly', probability=True)),
                             ('svm_rb', SVC(probability=True))],
                 n_jobs=-1)

In [211]:
vc_soft_pred = vc_soft.predict(X_test)
vc_hard_pred = vc_hard.predict(X_test)

In [212]:
print("voting classifier prediction, soft,classification report",classification_report(vc_soft_pred,y_test))
print("voting classifier prediction, hard,classification report",classification_report(vc_hard_pred,y_test))

voting classifier prediction, soft,classification report               precision    recall  f1-score   support

           B       0.99      0.96      0.97        73
           M       0.93      0.98      0.95        41

    accuracy                           0.96       114
   macro avg       0.96      0.97      0.96       114
weighted avg       0.97      0.96      0.97       114

voting classifier prediction, hard,classification report               precision    recall  f1-score   support

           B       0.99      0.97      0.98        72
           M       0.95      0.98      0.96        42

    accuracy                           0.97       114
   macro avg       0.97      0.97      0.97       114
weighted avg       0.97      0.97      0.97       114



In [213]:
print("accuracy score, voting classifier, soft",accuracy_score(vc_soft_pred,y_test))
print("accuracy score, voting classifier, hard",accuracy_score(vc_hard_pred,y_test))

accuracy score, voting classifier, soft 0.9649122807017544
accuracy score, voting classifier, hard 0.9736842105263158
